In [1]:
import pandas as pd
import numpy as np
import json

Chargement des données initiales : 

In [33]:
data=pd.read_csv('1850_cleaned_global_temperature_by_country.csv')
#print(data.iloc[:,1:].max().max())
#print(data.iloc[:,1:].min().min())
len(data.columns)

1966

In [2]:
data=pd.read_csv('GlobalLandTemperaturesByCountry.csv',
                 usecols=['dt', 'AverageTemperature', 'Country'],
                 dtype={'dt': np.str, 'AverageTemperature': np.float, 'Country': np.str})
print(len(data))
data.head()

577462


,dt,AverageTemperature,Country
0,1743-11-01,4.384,Åland
1,1743-12-01,NaN,Åland
2,1744-01-01,NaN,Åland
3,1744-02-01,NaN,Åland
4,1744-03-01,NaN,Åland


Renommage des colonnes : 

In [3]:
data.columns = ['date', 'temp_moy', 'pays']
# data = data.dropna(axis=0)
data.head()

,date,temp_moy,pays
0,1743-11-01,4.384,Åland
1,1743-12-01,NaN,Åland
2,1744-01-01,NaN,Åland
3,1744-02-01,NaN,Åland
4,1744-03-01,NaN,Åland


On garde uniquement les pays dont on dispose de coordonnées géographiques : 

In [4]:
all_pays = data.pays.unique()
len(all_pays)

243

In [5]:
with open('world_countries.json', 'r') as read_file:
    data_map = json.load(read_file)
pays_map = set()
for feature in data_map['features']:
    pays_map.add(feature['properties']['name'])
len(pays_map)

177

In [6]:
filtered_countries = {pays for pays in all_pays if pays in pays_map}
len(filtered_countries)

170

In [7]:
print([pays for pays in pays_map if pays not in filtered_countries])

['Democratic Republic of the Congo', 'Brunei', 'Vanuatu', 'South Sudan', 'Northern Cyprus', 'Somaliland', 'Kosovo']


In [8]:
data = data[data.pays.isin(filtered_countries)]
len(data)

405074

On effectue la moyenne des températures entre 1850 et 1900, pour chaque mois et chaque pays. On remplace la température moyenne par l'écart par rapport à la température moyenne sur 1850-1900 : 

In [9]:
data_1850_1900 = data[data.date <= '1899-12-01']
data_1850_1900 = data_1850_1900[data_1850_1900.date >= '1850-01-01']
data_1850_1900 = data_1850_1900.dropna(axis=0)
data_1850_1900.head()

,date,temp_moy,pays
3380,1850-01-01,-1.288,Afghanistan
3381,1850-02-01,3.143,Afghanistan
3382,1850-03-01,8.583,Afghanistan
3383,1850-04-01,13.456,Afghanistan
3384,1850-05-01,18.717,Afghanistan


In [10]:
dict_country_temp_moys = {} # dictionnaire pays => [temps moyennes / mois sur 1850-1900]
for country in filtered_countries:
    cur_data_country = data_1850_1900[data_1850_1900.pays == country]
    dict_country_temp_moys[country] = []
    for i in range(1,13):
        cur_data = cur_data_country[cur_data_country.date.str.contains('-{:02}-'.format(i))]
        temps_moy_country_month = cur_data.temp_moy.mean()
        dict_country_temp_moys[country].append(temps_moy_country_month)

On garde les relevés supérieurs à 1850 : 

In [11]:
# pays qui n'ont pas de relevés avant 1850 : 
x = data[data.date <= '1850-01-01'].pays.values
[pays for pays in filtered_countries if pays not in x]

['Nigeria',
 'Swaziland',
 'Lesotho',
 'Eritrea',
 'New Zealand',
 'Argentina',
 'Cameroon',
 'Solomon Islands',
 'Angola',
 'Antarctica',
 'New Caledonia',
 'Sudan',
 'Central African Republic',
 'Philippines',
 'Australia',
 'Yemen',
 'Gabon',
 'Botswana',
 'Namibia',
 'Chad',
 'East Timor',
 'Chile',
 'Bolivia',
 'Fiji',
 'Falkland Islands',
 'Papua New Guinea',
 'South Africa',
 'Equatorial Guinea',
 'Costa Rica',
 'Peru',
 'Ecuador',
 'French Southern and Antarctic Lands',
 'Djibouti']

In [12]:
data = data[data.date >= '1850-01-01']
print(len(data))
data.head()

328104


,date,temp_moy,pays
3380,1850-01-01,-1.288,Afghanistan
3381,1850-02-01,3.143,Afghanistan
3382,1850-03-01,8.583,Afghanistan
3383,1850-04-01,13.456,Afghanistan
3384,1850-05-01,18.717,Afghanistan


On remplace la température moyenne par l'écart par rapport à la moyenne 1850-1900 :

On transforme la forme du dataset : colonnes = nom_pays, date_1, date_2, ...

In [13]:
dates = data.date.unique()
cols = np.concatenate((['pays'], dates))
content = []
months = [int(date[5:7]) for date in dates]

content = []
for country in filtered_countries:
    print(country)
    row = [country]
    data_country = data[data.pays == country]
    for i in range(len(dates)):
        temp_moy = dict_country_temp_moys[country][months[i]-1]
        if (dates[i] in data_country.date.values):
            row.append(data_country[data_country.date == dates[i]].temp_moy.values[0] - temp_moy)
        else:
            row.append(float('nan'))
    content.append(row)

nouv_data =  pd.DataFrame(content, columns=cols)
nouv_data.head()

Iran
Iraq
Cuba


KeyboardInterrupt: 

On enregistre le dataset filtré et transformé : 

In [27]:
nouv_data.to_csv('1850_cleaned_global_temperature_by_country.csv', index=False)